In [4]:
import cv2
import pytesseract
from ultralytics import YOLO
import sqlite3
import config  # 좌석 좌표 데이터가 저장된 파일 [cite: 5]
import os
import re

# ==========================================
# 블록 1: 헤더 및 초기 설정 (Header Block)
# ==========================================
def step0_initialize():
    # Tesseract 실행 파일 경로 설정
    pytesseract_path = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
    # 0. 사전 준비 단계 (Initialization)
    if os.path.exists(pytesseract_path):
        pytesseract.pytesseract.tesseract_cmd = pytesseract_path
        print("Tesseract 로드완료")
        # 딥러닝 모델 로드 [cite: 5]
        model_b = YOLO('yolov8n.pt')
        print("YOLO 로드완료")
        return pytesseract, model_b
    else:
        print("Tesseract 설치 경로를 찾을 수 없습니다.")
        return False

# ==========================================
# 블록 2: 기능별 함수 블록 (Function Blocks)
# ==========================================

def step1_ocr_process(roi_frame):
    """
    1. 입력 및 인식 단계 [cite: 1]
    - 전처리(흑백, 블러, 이진화, 반전) 후 Tesseract로 번호판 추출 [cite: 2, 6]
    """
    # 의사코드의 전처리 과정: Gray -> Blur -> Otsu -> Not [cite: 2, 6]
    gray = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    processed_img = cv2.bitwise_not(thresh)
    cv2.imshow("Step 1: OCR View", processed_img)

    # Tesseract 분석(kor+eng를 추가하므로써 한글이 숫자로 오인되는 경우 방지) [cite: 2, 6]
    car_id_raw = pytesseract.image_to_string(processed_img, lang='kor+eng', config='--psm 7')
    # 한글과 숫자만 남기고 공백, 특수문자 모두 제거
    car_id = re.sub(r'[^0-9가-힣]', '', car_id_raw)
    return car_id.strip()

# ==========================================
# 블록 3: 메인 제어 루프 (Main Block) [cite: 11]
# ==========================================
def main():
    cap = cv2.VideoCapture(0)
    initialize = step0_initialize()

    cv2.namedWindow('Parking System')
    cv2.setMouseCallback('Parking System', mouse_control)

if __name__ == "__main__":
    main()

Tesseract 로드완료
YOLO 로드완료

[인식 결과]
차량 번호: 
